In [94]:
# Imports
import bs4
import requests
import selenium
import re
import urllib.request
import csv

# Henter en url og bruger requests til at fejl finde eller raise en fejl.
# BeautifulSoup parser html og laver det til læseligt tekst.
url = "https://www.kongehuset.dk/monarkiet-i-danmark/nytarstaler/hendes-majestat-dronningens-nytarstaler"
r_url = requests.get(url)
r_url.raise_for_status()
soup = bs4.BeautifulSoup(r_url.text, 'html.parser')

# Hent a href links og skriver indholdet til csv filer som en liste.
def get_links_and_write_to_csv(output_file):
    list_of_links = [] 
      
    # Henter alle a href links og adder det til en liste.
    for link in soup.findAll('a', attrs={'href': re.compile("https://www.kongehuset.dk")}):
        list_of_links.append(link.get('href'))
        #print(link.get('href'))
       
    #Skriv indholdet af linksnes tekst til en csv fil.
    with open(output_file, 'w') as file_object:
        write_to = csv.writer(file_object, delimiter=',')
        write_to.writerow(list_of_links)         
    return list_of_links
    
get_links_and_write_to_csv("Links.csv")

# Tager en fil og add'er indholdet af filen til en liste, som den returnere.
def read_csv_and_add_to_list(input_file): 
    list_of_urls = []
    with open(input_file, 'r') as file_object:
        read_it = csv.reader(file_object, delimiter=',')
        for row in read_it:
            list_of_urls.append(row)    
    return(list_of_urls)
                       
read_csv_and_add_to_list("Links.csv")  

# Åbner links og læser indholdet på de links og tilføjer indholdet til csv filer.
def open_links(links):
    for link in links:
        response = urllib.request.urlopen(link) # (link, data, timeout) ?????????
        #file_name = link.replace('https://www.', '').replace('.dk', '')
        #print(file_name)
        #with open(file_name, 'w') as out_file:
            #shutil.copyfileobj(response, out_file)
                              
open_links(read_csv_and_add_to_list("Links.csv"))

AttributeError: 'list' object has no attribute 'timeout'